In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from pytexit import py2tex
from IPython.display import Markdown
from gekko import GEKKO
import pandas as pd

%matplotlib widget

In [4]:
# Import 'measured data'
HIV_Data = pd.read_excel('hiv_data.xlsx')

HIV_Data['Virus'] = np.power(10,HIV_Data.lv)
print(HIV_Data)
plt.figure()
plt.plot(HIV_Data.time,HIV_Data.lv,'.')
plt.grid()
plt.title('HIV Data (log(x))')
plt.xlabel('Time (yrs)')
plt.ylabel('Virus Count (log scale)')

plt.figure()
plt.semilogy(HIV_Data.time,HIV_Data.Virus,'.')
plt.grid()
plt.title('HIV Data')
plt.xlabel('Time (yrs)')
plt.ylabel('Virus Count (log scale)')


     time    lv         Virus
0    0.00  1.20  1.584893e+01
1    0.25  1.67  4.677351e+01
2    0.50  2.06  1.148154e+02
3    0.75  2.05  1.122018e+02
4    1.00  3.57  3.715352e+03
..    ...   ...           ...
56  14.00  6.42  2.630268e+06
57  14.25  7.09  1.230269e+07
58  14.50  7.37  2.344229e+07
59  14.75  6.56  3.630781e+06
60  15.00  6.69  4.897788e+06

[61 rows x 3 columns]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Virus Count (log scale)')

In [15]:
# Exercise Problem 4
# https://apmonitor.com/pdc/index.php/Main/SimulateHIV

######################### M1 #################################
HIVmodel = GEKKO()    # create GEKKO model for dx/dt
HIVmodel.time = np.linspace(0,15,301) # time points

# dt is in the units of YEARS.

kr1 = 1e5  # = new healthy cells per year
kr2 = 0.1  # = death rate of healthy cells
kr3 = 2e-7 # = healthy cells converting to infected cells
kr4 = 0.5  # = death rate of infected cells
kr5 = 5    # = death rate of virus
kr6 = 100  # = production of virus by infected cells

Healthy  = HIVmodel.Var(1e6) # GEKKO variable = healthy cells = 1,000,000 initially
Infected = HIVmodel.Var(0) # GEKKO variable = infected cells = 0 initially
Virus    = HIVmodel.Var(100) # GEKKO variable = virus = 100 initially

HIVmodel.Equation(Healthy.dt()  == (kr1) - (kr2 * Healthy)  - (kr3 * Healthy * Virus) )
HIVmodel.Equation(Infected.dt() == ( kr3 * Healthy * Virus) - (kr4 * Infected) )
HIVmodel.Equation(Virus.dt()    == (-kr3 * Healthy * Virus) - (kr5 * Virus) + (kr6 * Infected) )

# solve ODEs and plot
HIVmodel.options.IMODE = 4

# Type: Integer, Input/Output
# Default Value: 0
# Description: Cold start model: 0=warm start, 1=cold start, 2=decompose problem
HIVmodel.options.COLDSTART = 2

#HIVmodel.options.SOLVER = 1
#HIVmodel.options.MAX_ITER = 1000

#---------------------
HIVmodel.options.NODES = 3
#---------------------

HIVmodel.options.TIME_SHIFT=0
HIVmodel.solve(disp=False)


plt.figure()
plt.semilogy(HIVmodel.time,Healthy,'b',marker='',linewidth=1,label='Healthy Cells')
plt.semilogy(HIVmodel.time,Infected,'g:',marker='',linewidth=1,label='Infected Cells')
plt.semilogy(HIVmodel.time,Virus,'r--',marker='',linewidth=1,label='Virus')

plt.semilogy(HIV_Data.time,HIV_Data.Virus,'.', color = 'red',label = 'Data To Match')

plt.grid(True)

plt.title('HIV Simulation')
plt.xlabel('time')
plt.ylabel('y(t)')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …